# Cleaning data

In [167]:
import pandas as pd
import numpy as np

columns = ['Date',
          'Tournament',
          'Surface',
          'Round',
          'Best of',
          'Winner',
          'Loser',
          'WRank',
          'LRank',
          'WPts',
          'LPts',
          'Comment',
          'MaxW',
          'MaxL',
          'AvgW',
          'AvgL']


for year in range(2011, 2019):
    X = pd.read_excel('Data\\' + str(year) +'.xlsx')
    X_prev = pd.read_excel('Data\\' + str(year-1) +'.xlsx')
    X = X[X['Comment'] == 'Completed']
    X = X[columns]
    winner = np.random.randint(0,2,len(X))
    looser = 1 - winner
    X['Winner_id'] = winner
    for id_player in range(2):
        X['player_' + str(id_player)] = np.where(X['Winner_id']==id_player, X['Winner'], X['Loser'])
        X['rank_player_' + str(id_player)] = np.where(X['Winner_id']==id_player, X['WRank'], X['LRank'])
        X['points_player_' + str(id_player)] = np.where(X['Winner_id']==id_player, X['WPts'], X['LPts'])
        X['max_player_' + str(id_player)] = np.where(X['Winner_id']==id_player, X['MaxW'], X['MaxL'])
        X['avg_player_' + str(id_player)] = np.where(X['Winner_id']==id_player, X['AvgW'], X['AvgL'])
    X.drop(['Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts', 'Comment', 'MaxW', 'MaxL', 'AvgW', 'AvgL'], axis=1, inplace=True)
    X_prev = X_prev[X_prev['Comment'] == 'Completed']

    X['success_player_0'] = 0
    X['success_player_1'] = 0
    X['success_surface_player_0'] = 0
    X['success_surface_player_1'] = 0
    X['best_rank_player_0'] = np.nan
    X['best_rank_player_1'] = np.nan
    for index, row in X.iterrows():
        for id_player in range(2):
            victories = X_prev[X_prev['Winner']==row['player_' + str(id_player)]]
            defeats = X_prev[X_prev['Loser']==row['player_'  + str(id_player)]]
            victories_surface = victories[victories['Surface']==row['Surface']]
            defeats_surface = defeats[defeats['Surface']==row['Surface']]
            if len(victories) + len(defeats) > 0:
                X.loc[index, 'success_player_' + str(id_player)] = len(victories) / (len(victories) + len(defeats))
                best_rank = np.nan
                if len(victories) > 0:
                    best_rank = np.min(victories['WRank'])
                if len(defeats) >0:
                    best_rank = min(best_rank, np.min(defeats['LRank']))
                X.loc[index, 'best_rank_player_' + str(id_player)] = best_rank
            if len(victories_surface) + len(defeats_surface) > 0:
                X.loc[index, 'success_surface_player_' + str(id_player)] = len(victories_surface) / (len(victories_surface) + len(defeats_surface))
            
    if year == 2011:
        X_all = X
    else:
        X_all = X_all.append(X, ignore_index=True)
    print('Year ' + str(year) + ' done.')    
    
X_all.head()

Year 2011 done.
Year 2012 done.
Year 2013 done.
Year 2014 done.
Year 2015 done.
Year 2016 done.
Year 2017 done.
Year 2018 done.


,Date,Tournament,Surface,Round,Best of,Winner_id,player_0,rank_player_0,points_player_0,max_player_0,...,rank_player_1,points_player_1,max_player_1,avg_player_1,success_player_0,success_player_1,success_surface_player_0,success_surface_player_1,best_rank_player_0,best_rank_player_1
0,2011-01-02,Brisbane International,Hard,1st Round,3,0,Lopez F.,32.0,1300.0,1.67,...,57.0,839.0,2.60,2.36,0.521739,0.538462,0.482759,0.500000,22.0,37.0
1,2011-01-02,Brisbane International,Hard,1st Round,3,1,De Bakker T.,43.0,975.0,1.85,...,40.0,1031.0,2.10,1.99,0.521739,0.518519,0.466667,0.515152,43.0,39.0
2,2011-01-03,Brisbane International,Hard,1st Round,3,0,Berrer M.,58.0,835.0,2.01,...,75.0,643.0,1.91,1.82,0.413043,0.451613,0.459459,0.476190,42.0,40.0
3,2011-01-03,Brisbane International,Hard,1st Round,3,0,Kubot L.,70.0,695.0,1.91,...,104.0,541.0,2.16,1.94,0.368421,0.296296,0.333333,0.250000,41.0,68.0
4,2011-01-03,Brisbane International,Hard,1st Round,3,0,Mayer F.,37.0,1128.0,1.44,...,208.0,239.0,3.15,2.88,0.567568,0.250000,0.520000,0.250000,39.0,218.0


## Building model

We now have the following features :

- Best of : 2 or 3 sets game
- rank_player_x : ATP rank before match of player x
- points_player_x : ATP rank points before match of player x
- success_player_x : % of success of player x during last season
- success_surface_player_x : % of success of player x on this surface during last season
- best_rank_player_x : best ATP rank of player x last year

We have average quotes fromdifferent bookmakers.

We can expect the average of bookmakers quotes to be a very good predictor for match results are they reflect market implied probabilities.
Let's look if we can build a better predictor.

In [183]:
X = X_all.copy()
X.dropna(inplace=True)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

features = ['Best of', 
           'rank_player_0',
           'points_player_0',
           'rank_player_1',
           'points_player_1',
           'success_player_0',
           'success_player_1',
           'success_surface_player_0', 
           'success_surface_player_1',
           'best_rank_player_0',
           'best_rank_player_1']

#Model
model = LogisticRegression()

#Cross validation
y = X['Winner_id']
scores = cross_val_score(model, X[features], y, scoring='accuracy', cv=5)
print('cross_validation score : {score:.2f}, '.format(score=scores.mean()))

#Dumb predictions
rank_predictions = np.where(X['rank_player_0'] >= X['rank_player_1'], 1, 0)
cotes_predictions = np.where(X['avg_player_0'] >= X['avg_player_1'], 1, 0)

test_score_rank = accuracy_score(y, rank_predictions)
test_score_cotes = accuracy_score(y, cotes_predictions)
print('score rank : {rank:.2f}, score cotes : {cotes:.2f}'.format(rank=test_score_rank, cotes=test_score_cotes))

cross_validation score : 0.67, 
score rank : 0.66, score cotes : 0.70
